In [ ]:
# Get retention data
names = ["addr", "time", "lvl"]
data = pd.read_csv("../data/retention.csv.gz", delimiter='\t', names=names)
data["lvl"] = data["lvl"].str.strip('[]').str.split(', ')
data = data.explode("lvl", ignore_index=True)
data["lvl"] = pd.to_numeric(data["lvl"])
data["time"] -= data.groupby("addr")["time"].transform("first") 
data["lvli"] = data.groupby("addr")["lvl"].transform("first")
data["i"] = data.index % 16
data["tmin"] = data["addr"] % 60
data["tmax"] = data["addr"] % 60 + 4
data

In [ ]:
# Get target times
tdata = []
times = [0, 0.1, 1, 10, 100, 1000, 10000, 100000]
for time in times:
    idx = (data["time"] - time).abs().groupby([data["addr"], data["i"]]).idxmin()
    d = data[data.index.isin(idx)]
    d = d[(d["time"] <= time*1.2) & (d["time"] >= time*0.8)]
    print(time, len(d))
    d["timept"] = time
    tdata.append(d)
tdata = pd.concat(tdata)
tdata.to_csv("data/retention.min.csv.gz")
tdata